In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/DEIMOS_data/J1251_data/*'

In [3]:
file_list = []
out_file_det2 = []
out_file_det6 = []

for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'J1251' in p:
        file_list.append(p)
        f = p.split('_')
        frame = f[-4]+ '_' + f[-3][:4]
        out_file_det2.append('J1251_DEIMOS_spectra/J1251_deimos_det2_'+ frame + '_clean.fits')
        out_file_det6.append('J1251_DEIMOS_spectra/J1251_deimos_det6_'+ frame + '_clean.fits')

In [12]:
file_list = sorted(file_list)
file_list

['../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0079-J1251m1_DEIMOS_2021Jun13T075045.110.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0614_0071-J1251m1_DEIMOS_20210614T060232.755.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0615_0073-J1251m2_DEIMOS_20210615T060533.763.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0615_0074-J1251m2_DEIMOS_20210615T063713.526.fits']

In [5]:
out_file_det2

['J1251_DEIMOS_spectra/J1251_deimos_det2_d0613_0078_clean.fits',
 'J1251_DEIMOS_spectra/J1251_deimos_det2_d0614_0071_clean.fits',
 'J1251_DEIMOS_spectra/J1251_deimos_det2_d0613_0079_clean.fits',
 'J1251_DEIMOS_spectra/J1251_deimos_det2_d0615_0074_clean.fits',
 'J1251_DEIMOS_spectra/J1251_deimos_det2_d0615_0073_clean.fits']

### finding out spatial location of 2d lens spectra in science image¶

In [8]:
file_list_spec1d = []
frame_list = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec1d' in p and 'J1251' in p:
        file_list_spec1d.append(p)
        f = p.split('_')
        frame = f[-4]+ '_' + f[-3][:4]
        frame_list.append(frame)

In [9]:
file_list_spec1d

['../galaxy_spectra/DEIMOS_data/J1251_data/spec1d_d0614_0071-J1251m1_DEIMOS_20210614T060232.755.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec1d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec1d_d0615_0074-J1251m2_DEIMOS_20210615T063713.526.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec1d_d0615_0073-J1251m2_DEIMOS_20210615T060533.763.fits',
 '../galaxy_spectra/DEIMOS_data/J1251_data/spec1d_d0613_0079-J1251m1_DEIMOS_2021Jun13T075045.110.fits']

In [11]:
for fl, frm in zip(file_list_spec1d, frame_list):
    sp = pf.open(fl)
    #m = sp.slitinfo['objid']=='lens'
    nspec = sp[0].header['NSPEC']
    print(frm, nspec)
    #if frm =='d0613_0105':
        #pass
    #elif frm =='d0613_0106':
        #pass
    #else:
    for i in range(nspec):
        if 'lens' in sp[i+1].header['HIERARCH MASKDEF_OBJNAME']:
            print(sp[i+1].header['NAME'])

d0614_0071 107
SPAT1204-SLIT1203-DET02
SPAT1198-SLIT1198-DET06
d0613_0078 106
SPAT1208-SLIT1203-DET02
SPAT1202-SLIT1198-DET06
d0615_0074 117
SPAT1140-SLIT1136-DET02
SPAT1134-SLIT1131-DET06
d0615_0073 116
SPAT1140-SLIT1136-DET02
SPAT1134-SLIT1131-DET06
d0613_0079 108
SPAT1209-SLIT1203-DET02
SPAT1202-SLIT1198-DET06


## median image after sky subtraction

In [ ]:
78, 2 - [1150, 1250]; 78,6 - [1145, 1250]; 79,2 - [1150, 1250]; 79, 6- [1145, 1250]
71, 2 - [1150, 1250]; 71,6 - [1145, 1250]; 
73, 2 - [1084, 1184]; 73,6 - [1075, 1180]; 74, 2 - [1084, ,1184]; 74, 6 - [1075, 1180]

In [13]:
xtrim_det2 = [[1150, 1250], [1084, 1184]]
xtrim_det6 = [[1145, 1250], [1075, 1180]]
spec_det_2 = []
spec_det_6 = []
for i, fl in enumerate(file_list):
    if i<3:
        spec1 = spec2d.Spec2d(fl, hext=14, xtrim=xtrim_det2[0])
        spec2 = spec2d.Spec2d(fl, hext=66, xtrim=xtrim_det6[0])
    else:
        spec1 = spec2d.Spec2d(fl, hext=14, xtrim=xtrim_det2[1])
        spec2 = spec2d.Spec2d(fl, hext=66, xtrim=xtrim_det6[1])
    
    spec1.set_dispaxis('y')
    spec1.subtract_sky_2d(use_skymod=True)
    spec_det_2.append(spec1.data)
    
    spec2.set_dispaxis('y')
    spec2.subtract_sky_2d(use_skymod=True)
    spec_det_6.append(spec2.data)


Loading file ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits (HDU=14)
The input dataset was trimmed
 xrange: 1150:1251.  yrange: 0:4096
Final data dimensions (x y): 101 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 101


Loading file ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits (HDU=66)
The input dataset was trimmed
 xrange: 1145:1251.  yrange: 0:4096
Final data dimensions (x y): 106 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 106


Old value of dispaxis: x

Dispers

In [14]:
median_im_det2 = np.median(np.asarray(spec_det_2), axis=0)
median_im_det6 = np.median(np.asarray(spec_det_6), axis=0)

In [15]:
pf.PrimaryHDU(median_im_det2).writeto('J1251_DEIMOS_spectra/J1251_median_image_det2.fits')
pf.PrimaryHDU(median_im_det6).writeto('J1251_DEIMOS_spectra/J1251_median_image_det6.fits')

### subtract sky and cosmic rays

In [16]:
extension = [14, 66]

median_im = ['J1251_DEIMOS_spectra/J1251_median_image_det2.fits', 
             'J1251_DEIMOS_spectra/J1251_median_image_det6.fits']
i = 0
for fl, ofl1, ofl2 in zip(file_list, out_file_det2, out_file_det6):
    if i<3:
        xtrim_list = [[1150, 1250], [1145, 1250]]
    else:
        xtrim_list = [[1084, 1184], [1075, 1180]]
        
    for ext, xtr, mdim in zip(extension, xtrim_list, median_im):
        print(i, ext, xtr, mdim)
        spec = spec2d.Spec2d(fl, hext=ext, xtrim=xtr)
        #spec2 = spec2d.Spec2d(fl, hext=66, xtrim=[970, 1080])
        spec.set_dispaxis('y')
        spec.subtract_sky_2d(use_skymod=True)
        spec.szap_new(infile=mdim)
    
        filled_data = spec.data
        mask = spec.szap_mask
        #print(filled_data.shape)
        ivar_im = pf.open(fl)[ext+1].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
        sky_im = pf.open(fl)[ext+2].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
        wav_im = pf.open(fl)[ext+7].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax]
    
        ivar_im[mask] = 1.e-5
        #print(ivar_im.shape)
        
        prim_hdr = pf.open(fl)[0].header
        prim_hdr.append(('OBJECT', 'J1251+2935'))
        prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
        phdu = pf.PrimaryHDU(header=prim_hdr)
        
        sci_hdr = pf.Header()
        sci_hdr.append(('OBJECT', 'J1251+2935'))
        sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
        sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')

        ivar_hdr = pf.Header()
        ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
        ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')

        sky_hdr = pf.Header()
        sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
        sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')

        wav_hdr = pf.Header()
        wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
        wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')

        mask_hdr = pf.Header()
        mask_hdr.append(('IMTYPE', 'MASK'))
        cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
            'location from median image'
        mask_hdr.append(('', cmt))
        mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')

        hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
        if ext==14:
            hdul.writeto(ofl1)
        else:
            hdul.writeto(ofl2)
    i += 1

0 14 [1150, 1250] J1251_DEIMOS_spectra/J1251_median_image_det2.fits

Loading file ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0613_0078-J1251m1_DEIMOS_2021Jun13T071903.274.fits (HDU=14)
The input dataset was trimmed
 xrange: 1150:1251.  yrange: 0:4096
Final data dimensions (x y): 101 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 101


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used
0 66 [1145, 1250] J1251_DEIMOS_spectra/J1251_median_image_det6.fits

Loa


From now cosmic ray rejected data will be used
4 66 [1075, 1180] J1251_DEIMOS_spectra/J1251_median_image_det6.fits

Loading file ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0615_0074-J1251m2_DEIMOS_20210615T063713.526.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1251_data/spec2d_d0615_0074-J1251m2_DEIMOS_20210615T063713.526.fits (HDU=66)
The input dataset was trimmed
 xrange: 1075:1181.  yrange: 0:4096
Final data dimensions (x y): 106 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 106


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used
